In [1]:
from skimage.feature import hog
from skimage.transform import pyramid_gaussian
from skimage.io import imread
#from sklearn.externals
import joblib
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import classification_report
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
from skimage import color
from imutils.object_detection import non_max_suppression
import imutils
import numpy as np
import argparse
import cv2
import os
import glob
from PIL import Image # This will be used to read/modify images (can be done via OpenCV too)
from numpy import *

In [2]:
# define parameters of HOG feature extraction
orientations = 3
pixels_per_cell = (16, 16)
cells_per_block = (2, 2)

In [3]:
# define path to images:

pos_im_path = r"/media/dtu-project2/2GB_HDD/Detection_HOG_SVM/bigger_dataset_128_64/pos"  
neg_im_path = r"/media/dtu-project2/2GB_HDD/Detection_HOG_SVM/bigger_dataset_128_64/neg"

In [4]:
# read the image files:
pos_im_listing = os.listdir(pos_im_path) # it will read all the files in the positive image path (so all the required images)
neg_im_listing = os.listdir(neg_im_path)
num_pos_samples = size(pos_im_listing) # simply states the total no. of images
num_neg_samples = size(neg_im_listing)
print(num_pos_samples) # prints the number value of the no.of samples in positive dataset
print(num_neg_samples)


2622
2448


In [5]:
# compute HOG features and label them:
data= []
labels = []
for file in pos_im_listing: #this loop enables reading the files in the pos_im_listing variable one by one
    filename = os.path.join(pos_im_path, file)
    #print(file)
    img = Image.open(filename)
    
    #img = img.resize((240,240))
    #img = img.resize((64,128))
    gray = img.convert('L') # convert the image into single channel i.e. RGB to grayscale
    # calculate HOG for positive features
    fd = hog(gray, orientations, pixels_per_cell, cells_per_block, block_norm='L2', feature_vector=True)# fd= feature descriptor
    #print(fd.shape)
    data.append(fd)
    #print(size(data))
    labels.append(1)
    #print(labels)

## for negative images
for file in neg_im_listing: #this loop enables reading the files in the pos_im_listing variable one by one
    filename = os.path.join(neg_im_path, file)
    #print(file)
    img = Image.open(filename)
    #img=Image.open(neg_im_path + '\\' + file)
    #img = img.resize((240,240))
    #img = img.resize((64,128))
    gray= img.convert('L')
    # Now we calculate the HOG for negative features
    fd = hog(gray, orientations, pixels_per_cell, cells_per_block, block_norm='L2', feature_vector=True) 
    data.append(fd)
    labels.append(0)



In [6]:
np_data = np.array(data)
np_data.shape

(5070, 588)

In [7]:
# encode the labels, converting them from strings to integers
le = LabelEncoder()
labels = le.fit_transform(labels) 
type(labels)
print(labels.shape)

(5070,)


In [8]:
#%%
# Partitioning the data into training and testing splits, using 80%
# of the data for training and the remaining 20% for testing
print(" Constructing training/testing split...")
(trainData, testData, trainLabels, testLabels) = train_test_split(
	np.array(data), labels, test_size=0.20, random_state=10, stratify=labels)
print('train data: \t', trainData.shape)
print('test data \t', testData.shape)

s=  np.sum(testLabels)
test_label_per = s*100/len(testLabels)
print(test_label_per)

s=  np.sum(trainLabels)
train_label_per = s*100/len(trainLabels)
print(train_label_per)

Constructing training/testing split...
train data: 	 (4056, 588)
test data 	 (1014, 588)
51.676528599605525
51.7258382642998


In [9]:
#%% Train the linear SVM
from sklearn.svm import LinearSVC, SVC
print(" Training Linear SVM classifier...")
model  = SVC(max_iter=1000)
model.fit(trainData, trainLabels)
print('training finished')


Training Linear SVM classifier...
training finished


In [10]:
#%% Evaluate the classifier######################################################################
print(" Evaluating classifier on test data ...")
predictions = model.predict(testData)
print(classification_report(testLabels, predictions))

Evaluating classifier on test data ...
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       490
           1       1.00      0.98      0.99       524

    accuracy                           0.99      1014
   macro avg       0.99      0.99      0.99      1014
weighted avg       0.99      0.99      0.99      1014



In [11]:
# Save the model:
#%% Save the Model
joblib.dump(model, 'model_name.npy')

['model_name.npy']

In [37]:
### visualizing model output for some of the images form test data
import os
import matplotlib.pyplot as plt 
%matplotlib inline
from sklearn.metrics import precision_score, recall_score, f1_score

extra__pos_data_dir = '/media/dtu-project2/2GB_HDD/Detection_HOG_SVM/neg_pos_imgs_for_hog_svm_unsen/pos'
extra__neg_data_dir = '/media/dtu-project2/2GB_HDD/Detection_HOG_SVM/neg_pos_imgs_for_hog_svm_unsen/neg'
pos_pred = []
neg_pred = []
test_data = []
positive_decision_boundary = 0
negative_decision_boundary = 0

for img in os.listdir(extra__pos_data_dir):
    file_path = os.path.join(extra__pos_data_dir, img)
    temp_img = Image.open(file_path)
    gray_img = temp_img.convert('L')
    #plt.figure()
    #plt.imshow(gray_img)
    fd,hog_feat = hog(gray_img, orientations, pixels_per_cell, cells_per_block, block_norm='L2', feature_vector=True,visualize=True)
    test_data.append(fd)
    #plt.figure()
    #plt.imshow(hog_feat)
    #plt.show()
    fd = np.reshape(fd,(1,fd.shape[0]))
    #model_out = model.predict(fd)
    model_dec_function = model.decision_function(fd)
    ### decision boundary
    if model_dec_function >= positive_decision_boundary:
        model_out = 1
    else:
        model_out = 0

    pos_pred.append(model_out)
    print(model_out, model_dec_function)

for img in os.listdir(extra__neg_data_dir):
    file_path = os.path.join(extra__neg_data_dir, img)
    temp_img = Image.open(file_path)
    gray_img = temp_img.convert('L')
    #plt.figure()
    #plt.imshow(gray_img)
    fd,hog_feat = hog(gray_img, orientations, pixels_per_cell, cells_per_block, block_norm='L2', feature_vector=True,visualize=True)
    test_data.append(fd)
    #plt.figure()
    #plt.imshow(hog_feat)
    #plt.show()
    fd = np.reshape(fd,(1,fd.shape[0]))
    #model_out = model.predict(fd)
    model_dec_function = model.decision_function(fd)
    ##### 
    if model_dec_function <= negative_decision_boundary:
        model_out = 0
    else:
        model_out = 1
    neg_pred.append(model_out)
    print(model_out, model_dec_function)

print(neg_pred)



1 [1.41729155]
1 [1.42386937]
1 [1.08206871]
1 [0.95133363]
1 [2.15899277]
1 [0.17348757]
1 [1.41917003]
1 [0.65989468]
1 [0.51310842]
1 [1.34118396]
1 [2.16472632]
1 [2.75392786]
1 [0.82352868]
1 [1.82503182]
1 [1.10183252]
1 [2.19164648]
1 [1.188263]
1 [0.72550977]
1 [0.27178254]
1 [1.02479749]
1 [2.28818252]
1 [1.53986095]
1 [1.79541675]
1 [1.75581883]
0 [-1.79034407]
0 [-0.70587294]
0 [-0.70408013]
0 [-1.0228518]
1 [0.01503621]
0 [-1.49849722]
0 [-0.22055954]
0 [-1.98241286]
1 [0.0372847]
0 [-0.94337344]
0 [-0.72238268]
0 [-2.06783902]
0 [-0.50435221]
0 [-1.28532151]
0 [-1.1096478]
0 [-0.10469809]
0 [-1.35522142]
0 [-0.70811752]
0 [-1.12773084]
0 [-1.12049859]
0 [-1.37299437]
0 [-1.1584465]
0 [-0.16713094]
0 [-1.65091858]
0 [-0.9832101]
0 [-1.54470685]
0 [-0.57097877]
0 [-1.79380405]
0 [-0.70202653]
0 [-1.23819027]
0 [-1.25206811]
0 [-1.23877065]
0 [-0.44437177]
0 [-0.74933115]
0 [-1.1491691]
0 [-1.5373227]
0 [-0.98781357]
0 [-1.59082349]
0 [-1.7413629]
0 [-1.1495025]
0 [-1.3427643

In [38]:
test_data = np.array(test_data)
test_data.shape
pred = pos_pred+neg_pred
pred = np.array(pred)
print(pred)

pos_label = [1]*len(pos_pred)
neg_label = [0]*len(neg_pred)
test_labels = np.array(pos_label+neg_label)
print(test_labels)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 1 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0]


In [12]:
print(" Evaluating classifier on test data ...")
predictions = model.predict(test_data)
print(classification_report(test_labels, pred))

Evaluating classifier on test data ...


NameError: name 'test_data' is not defined